In [1]:
from wdtools import *

In [5]:
# this will take about 7 minutes
start = time.time()
all_taxlot = combine_taxlot()
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 7.0 minutes to complete


In [3]:
all_taxlot.year.unique()

array(['2017', '2018', '2019', '2020', '2021', '2022'], dtype=object)

In [2]:
wd1 = combine_wd_tables(setID='Set001', nm_to_add=0)

In [3]:
wd2 = combine_wd_tables(setID='Set002', nm_to_add=1420)

In [12]:
wd1.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot'],
      dtype='object')

In [4]:
df1 = combined_reindexed_data(setID='Set001', nm_to_add=0) 

In [28]:
gdf1 = match_wd_data_with_taxlot(df=df1, setID='Set001', all_taxlot=all_taxlot, nm_to_add=0, export=True)

In [ ]:
def report_unmatched(gdf, setID, nm_to_add, mute = False):
    wd_df = combine_wd_tables(setID, nm_to_add)
    matched_rID = gdf.record_ID.unique()
    unmatched_wd_df = wd_df[~wd_df.record_ID.isin(matched_rID)]
    # Count the number of occurrences of each unique value in the ID column
    df = unmatched_wd_df
    IDcol = 'wetdet_delin_number'
    counts = df[IDcol].value_counts()
    # Sort the DataFrame based on the counts of each unique value in the ID column
    sorted_df = df.loc[df[IDcol].isin(counts.index)].sort_values(by=[IDcol])
    nc = unmatched_wd_df[unmatched_wd_df.parcel_id.astype(str) == 'nan'].shape[0]
    nr = round((nc/wd_df.shape[0]) * 100, 2)
    r = round((unmatched_wd_df.shape[0]/wd_df.shape[0]) * 100, 2)
    if not mute:
        print(f'it is about {r}% of data in the original {wd_df.shape[0]} records unmatched')
        print(f'there are {nc} records ({nr}% of the original records) without parcel id')
    return sorted_df

In [6]:
res1 = check_corrected_data(df=df1, setID='Set001', all_taxlot=all_taxlot, nm_to_add=0, export=True)

In [7]:
# matched records
len(res1[0].record_ID.unique())

1331

In [8]:
# matched records with original IDs (the difference with the above is the corrected records)
len(res1[0][~res1[0].wdID.isin(res1[1].wetdet_delin_number.values)].record_ID.unique())

1307

In [9]:
df_wlots_to_check1 = res1[3]

In [10]:
# unmatched records with parcel IDs
len(df_wlots_to_check1[df_wlots_to_check1.missinglot=='N'].record_ID.unique())

27

In [11]:
df_wlots_to_check1.to_csv(os.path.join(inpath + '\\output\\to_review\\', 'df_with_lots_check_Set001.csv'), index=False) 

In [12]:
df2 = combined_reindexed_data(setID='Set002', nm_to_add=1420) 

In [29]:
gdf2 = match_wd_data_with_taxlot(df=df2, setID='Set002', all_taxlot=all_taxlot, nm_to_add=1420, export=True)

In [13]:
res2 = check_corrected_data(df=df2, setID='Set002', all_taxlot=all_taxlot, nm_to_add=1420, export=True)

In [14]:
df_wlots_to_check2 = res2[3]

In [15]:
df_wlots_to_check2.to_csv(os.path.join(inpath + '\\output\\to_review\\', 'df_with_lots_check_Set002.csv'), index=False) 

In [16]:
# matched records
len(res2[0].record_ID.unique())

607

In [17]:
# matched records with original IDs (the difference with the above is the corrected records)
len(res2[0][res2[0].wdID.isin(res2[1].wetdet_delin_number.values)].record_ID.unique())

20

In [18]:
# unmatched records with parcel IDs
len(df_wlots_to_check2[df_wlots_to_check2.missinglot=='N'].record_ID.unique())

18

In [19]:
setgdf1 = gpd.read_file(os.path.join(inpath, 'GIS', 'Join_Statewide.gdb'), layer=f'WD_Set001_Combined')

In [20]:
setgdf2 = gpd.read_file(os.path.join(inpath, 'GIS', 'Join_Statewide.gdb'), layer=f'WD_Set002_Combined')

In [21]:
len(setgdf1.Record_ID.unique())

1362

In [22]:
missedmatch1 = compare_data_report(gdf=gpd.GeoDataFrame(res1[0], crs="EPSG:2992", geometry='geometry'), 
                                   setID='Set001', 
                                   nm_to_add=0, 
                                   export = True)

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\geodataframe.py:1327: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [23]:
# missed matches
len(missedmatch1[0])

23

In [24]:
# added matches
len(missedmatch1[2])

54

In [25]:
missedmatch2 = compare_data_report(gdf=gpd.GeoDataFrame(res2[0], crs="EPSG:2992", geometry='geometry'), 
                                   setID='Set002', 
                                   nm_to_add=1420, 
                                   export = True)

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [26]:
# missed matches
len(missedmatch2[0])

4

In [27]:
# added matches
len(missedmatch2[2])

15